<a href="https://colab.research.google.com/github/CristValen/Acciones-RNR/blob/main/nps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelBinarizer
from imblearn.over_sampling import SMOTE
import numpy as np
import pandas as pd

# Establecer la semilla del generador de números aleatorios
np.random.seed(42)

# Supongamos que 'df' es tu DataFrame y que 'clase' es tu variable objetivo
df = pd.read_csv('tu_archivo.csv')

# Crear las variables objetivo y características
X = df.drop('clase', axis=1)
y = df['clase']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Aplicar SMOTE al conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Crear el modelo de Random Forest
modelo = RandomForestClassifier(random_state=42)

# Definir la grilla de hiperparámetros para la búsqueda aleatoria
param_dist = {
    'n_estimators': [50, 100, 200, 300, 400],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Realizar la búsqueda aleatoria de hiperparámetros
busqueda_aleatoria = RandomizedSearchCV(estimator=modelo,
                                        param_distributions=param_dist,
                                        n_iter=100,
                                        cv=3,
                                        verbose=2,
                                        random_state=42,
                                        n_jobs=-1)

# Entrenar el modelo con los mejores hiperparámetros encontrados
busqueda_aleatoria.fit(X_train_res, y_train_res)

# Hacer predicciones en el conjunto de prueba
predicciones = busqueda_aleatoria.predict(X_test)

# Imprimir un informe de clasificación
print("Informe de clasificación:")
print(classification_report(y_test, predicciones))

# Imprimir la matriz de confusión
print("Matriz de confusión:")
print(confusion_matrix(y_test, predicciones))

# Imprimir la precisión del modelo
print("Precisión del modelo:")
print(accuracy_score(y_test, predicciones))

# Calcular e imprimir la puntuación AUC-ROC para cada clase
lb = LabelBinarizer()
lb.fit(y_test)
y_test_lb = lb.transform(y_test)
predicciones_lb = lb.transform(predicciones)
print("Puntuación AUC-ROC para cada clase:")
for i in range(len(lb.classes_)):
    print(f"{lb.classes_[i]}: {roc_auc_score(y_test_lb[:, i], predicciones_lb[:, i])}")



